In [10]:
pip install textBlob

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Here we attempt to use a python library called textblob which is mainly used for the transformation of text data. As we have identified, tweets commonly have many spelling and grammar mistakes that needs to be rectified.

In [72]:
from textblob import TextBlob
import pandas as pd
import numpy as np
import re
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import spacy
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package punkt to C:\Users\Josiah
[nltk_data]     Lee\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Josiah
[nltk_data]     Lee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
# Load the dataset
data = pd.read_csv('Twitter Sentiment/dataset.csv', header=None)
data.columns = ['labels', 'text']
print(len(data))
print(data.head())

100000
   labels                                               text
0       0  @switchfoot http://twitpic.com/2y1zl - Awww, t...
1       0  is upset that he can't update his Facebook by ...
2       0  @Kenichan I dived many times for the ball. Man...
3       0    my whole body feels itchy and like its on fire 
4       0  @nationwideclass no, it's not behaving at all....


TextBlob actually hs its own sentiment analysis method. So I tried my luck with it. The output of the polarity ranges from -1 to 1 and the huge number of 0s indicate that there is actually a lot of data that the model is unable to classify. Tired. 

In [19]:
predictions = []

for i in range(24999, 75000):
    string = data.text[i]
    tb = TextBlob(string)
    prob = tb.sentiment.polarity
    predictions.append(prob)

In [20]:
len(list(filter(lambda x: x==0, predictions)))

17789

In [16]:
data = data.iloc[25000:75000, :]
data

,Unnamed: 0,labels,text,cleaned_text,tokens,stemmed_tokens,lemmatized_token,filtered_tokens
25000,25000,0,@Eskimimi they didn't have any alcoholic ginge...,they didn t have any alcoholic ginger beer in...,"['they', 'didn', 't', 'have', 'any', 'alcoholi...","['they', 'didn', 't', 'have', 'ani', 'alcohol'...","['they', 'didn', 't', 'have', 'ani', 'alcohol'...","[alcoholic, ginger, beer, borough, yesterday, ..."
25001,25001,0,would like to settle down and watch the footba...,would like to settle down and watch the footba...,"['would', 'like', 'to', 'settle', 'down', 'and...","['would', 'like', 'to', 'settl', 'down', 'and'...","['would', 'like', 'to', 'settl', 'down', 'and'...","[would, like, settle, watch, football, beer, f..."
25002,25002,0,"@Popher lucky, it's deff a sub-zero temp for m...",lucky it s deff a sub zero temp for me in my ...,"['lucky', 'it', 's', 'deff', 'a', 'sub', 'zero...","['lucki', 'it', 's', 'deff', 'a', 'sub', 'zero...","['lucki', 'it', 's', 'deff', 'a', 'sub', 'zero...","[lucky, deff, sub, zero, temp, room]"
25003,25003,0,I dropped my USB drive in my coffee cup. Sudde...,i dropped my usb drive in my coffee cup sudden...,"['i', 'dropped', 'my', 'usb', 'drive', 'in', '...","['i', 'drop', 'my', 'usb', 'drive', 'in', 'my'...","['I', 'drop', 'my', 'usb', 'drive', 'in', 'my'...","[dropped, usb, drive, coffee, cup, suddenly, r..."
25004,25004,0,my thoughts go out to the family,my thoughts go out to the family,"['my', 'thoughts', 'go', 'out', 'to', 'the', '...","['my', 'thought', 'go', 'out', 'to', 'the', 'f...","['my', 'thought', 'go', 'out', 'to', 'the', 'f...","[thoughts, go, family]"
...,...,...,...,...,...,...,...,...
74995,74995,1,Yup. Going to sleep. Night guys.,yup going to sleep night guys,"['yup', 'going', 'to', 'sleep', 'night', 'guys']","['yup', 'go', 'to', 'sleep', 'night', 'guy']","['yup', 'go', 'to', 'sleep', 'night', 'guy']","[yup, going, sleep, night, guys]"
74996,74996,1,"@MsUnitedStates Thank you so, so, so, so much ...",thank you so so so so much for supporting us ...,"['thank', 'you', 'so', 'so', 'so', 'so', 'much...","['thank', 'you', 'so', 'so', 'so', 'so', 'much...","['thank', 'you', 'so', 'so', 'so', 'so', 'much...","[thank, much, supporting, us, tonight, looking..."
74997,74997,1,@tommcfly Hey Tom How are you? http://twitpic....,hey tom how are you comment please,"['hey', 'tom', 'how', 'are', 'you', 'comment',...","['hey', 'tom', 'how', 'are', 'you', 'comment',...","['hey', 'tom', 'how', 'be', 'you', 'comment', ...","[hey, tom, comment, please]"
74998,74998,1,Teasing and Mack 1-0 are taking Vegas by storm...,teasing and mack are taking vegas by storm wit...,"['teasing', 'and', 'mack', 'are', 'taking', 'v...","['teas', 'and', 'mack', 'are', 'take', 'vega',...","['tea', 'and', 'mack', 'be', 'take', 'vega', '...","[teasing, mack, taking, vegas, storm, another,..."


What I intend to do would be similar to what we previously had but instead I'll just add an autocorrect and see what happens.

In [25]:
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Remove mentions
    text = re.sub(r'#', '', text)  # Remove hashtags
    text = re.sub(r'[^A-Za-z]+', ' ', text)  # Keep only letters
    text = text.lower()  # Convert to lowercase
    return text

data['cleaned_text'] = data['text'].apply(clean_text)

In [33]:
blobs = []

for sentence in data.cleaned_text:
    blobs.append(TextBlob(sentence))

In [34]:
data['blobs'] = pd.Series(blobs)

In [90]:
correct_blobs = []

for blob in blobs[45000:55000]:
    correct_blobs.append(blob.correct())


correct_blobs

[TextBlob(" a that s not fun "),
 TextBlob("i had a dream about u u said u were ok amp let go that i did everything that i could to help i ll always have a place in my heart u "),
 TextBlob("omffffggggggg i gutta be up at and rotate is blasting her music "),
 TextBlob(" i miss do "),
 TextBlob(" it is true very unfortunate that happiness is a hard thing for many people in this world even children "),
 TextBlob("away man i can t believe that saturday is about to come to an end i have to to to school on monday "),
 TextBlob(" yesterday i dreamt i was being tortured "),
 TextBlob("i m bored at home right now "),
 TextBlob(" k great pick i m jealous freezing in sydney "),
 TextBlob("i wish i lived in america and had nicer legs "),
 TextBlob(" i m really upset about the whole situation since u sweet u should takeover their twitter act please ll"),
 TextBlob("ill post a sweet every half an hour to show that i m awake ugh i m dozing off "),
 TextBlob("remembers a time last monday when he coul

In [91]:
from nltk.tokenize import word_tokenize
tokenize_blobs = []
for blob in correct_blobs:
    tokenize_blobs.append(blob.words)

tokenize_blobs


[WordList(['a', 'that', 's', 'not', 'fun']),
 WordList(['i', 'had', 'a', 'dream', 'about', 'u', 'u', 'said', 'u', 'were', 'ok', 'amp', 'let', 'go', 'that', 'i', 'did', 'everything', 'that', 'i', 'could', 'to', 'help', 'i', 'll', 'always', 'have', 'a', 'place', 'in', 'my', 'heart', 'u']),
 WordList(['omffffggggggg', 'i', 'gutta', 'be', 'up', 'at', 'and', 'rotate', 'is', 'blasting', 'her', 'music']),
 WordList(['i', 'miss', 'do']),
 WordList(['it', 'is', 'true', 'very', 'unfortunate', 'that', 'happiness', 'is', 'a', 'hard', 'thing', 'for', 'many', 'people', 'in', 'this', 'world', 'even', 'children']),
 WordList(['away', 'man', 'i', 'can', 't', 'believe', 'that', 'saturday', 'is', 'about', 'to', 'come', 'to', 'an', 'end', 'i', 'have', 'to', 'to', 'to', 'school', 'on', 'monday']),
 WordList(['yesterday', 'i', 'dreamt', 'i', 'was', 'being', 'tortured']),
 WordList(['i', 'm', 'bored', 'at', 'home', 'right', 'now']),
 WordList(['k', 'great', 'pick', 'i', 'm', 'jealous', 'freezing', 'in', 'syd

In [68]:

pip install spacy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [92]:

import spacy
nlp = spacy.load("en_core_web_sm")
def lemmatize_words(word_list):
    doc = nlp(" ".join(word_list))  # Join the stemmed tokens into a string
    lemmatized_text = [token.lemma_ for token in doc]
    return lemmatized_text

lemma_blobs = list(map(lemmatize_words, tokenize_blobs))

In [93]:
stopped = []
stop_words = set(stopwords.words('english'))
stopped = list(map(lambda tokens: [word for word in tokens if word not in stop_words], tokenize_blobs))

In [94]:
# One hot encoding
categories = list(set([item for sublist in stopped for item in sublist]))
#print(categories)

vectorizer = CountVectorizer(binary=True, vocabulary=categories)
sentences = [' '.join(words) for words in stopped]

# Fit and transform the sentences
one_hot_encoded = vectorizer.fit_transform(sentences)

# Convert the one-hot encoded data to a DataFrame
encoded_df = pd.DataFrame(one_hot_encoded.toarray(), columns = categories)

# use the print function to check if the one hot encoding is correct
#print(encoded_df.iloc[954,:].sum(),sentences[954]) 
print(encoded_df)
#print(categories)

      cradle  gooodddd  watched  harassing  andrew  falls  branded  gotten  \
0          0         0        0          0       0      0        0       0   
1          0         0        0          0       0      0        0       0   
2          0         0        0          0       0      0        0       0   
3          0         0        0          0       0      0        0       0   
4          0         0        0          0       0      0        0       0   
...      ...       ...      ...        ...     ...    ...      ...     ...   
9995       0         0        0          0       0      0        0       0   
9996       0         0        0          0       0      0        0       0   
9997       0         0        0          0       0      0        0       0   
9998       0         0        0          0       0      0        0       0   
9999       0         0        0          0       0      0        0       0   

      ladytron  koolaid  ...  war  workman  heading  accounting

In [95]:
X_train, X_test, y_train, y_test=train_test_split(encoded_df, data.labels[45000:55000], stratify=data.labels[45000:55000], test_size=0.2, random_state=42)

In [83]:
print(y_train)
len(y_train)

50478    1
50488    1
49499    0
49605    0
50511    1
        ..
50240    1
49452    0
50248    1
50303    1
50529    1
Name: labels, Length: 1600, dtype: int64


1600

In [85]:
import sklearn

In [96]:
clf = sklearn.linear_model.LogisticRegression(random_state = 42).fit(X_train, y_train)


In [97]:
y_pred = clf.predict(X_test)

In [98]:
from sklearn.metrics import accuracy_score, classification_report
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Classification Report:\n{classification_rep}')

Accuracy: 0.737
Classification Report:
              precision    recall  f1-score   support

           0       0.76      0.70      0.73      1000
           1       0.72      0.78      0.75      1000

    accuracy                           0.74      2000
   macro avg       0.74      0.74      0.74      2000
weighted avg       0.74      0.74      0.74      2000

